In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline

In [3]:
from IPython.display import display

In [4]:
import os
import pandas as pd
import matplotlib.pyplot as plt

In [5]:
from datascifuncs.tidbit_tools import load_json, write_json, print_json, check_directory_name

In [6]:
main_dir = 'EmotionFaceClassifier'
check_directory_name(main_dir)

Directory set to /Users/dsl/Documents/GitHub/EmotionFaceClassifier, matches target dir string EmotionFaceClassifier.


True

In [7]:
from utils.decomposition_feature_extract import create_X_y

In [8]:
common_dicts = load_json('./configs/input_mappings.json')

In [9]:
emotion_colors = common_dicts['plotly_styles']['Training']['color']

In [10]:
# Read in FER 2013 data
fer2013_path = 'data/fer2013_paths.csv'
fer2013 = pd.read_csv(fer2013_path)

In [11]:
fer2013.head()

,emotion_id,pixels,Usage,emotion,image,usage,emo_count_id,img_path,color
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training,Angry,[[ 70 80 82 ... 52 43 41]\n [ 65 61 58 ...,Training,1,data/Training/Angry/Angry-1.jpg,red
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training,Angry,[[151 150 147 ... 129 140 120]\n [151 149 149 ...,Training,2,data/Training/Angry/Angry-2.jpg,red
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training,Fear,[[231 212 156 ... 44 27 16]\n [229 175 148 ...,Training,1,data/Training/Fear/Fear-1.jpg,slategray
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training,Sad,[[ 24 32 36 ... 173 172 173]\n [ 25 34 29 ...,Training,1,data/Training/Sad/Sad-1.jpg,blue
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training,Neutral,[[ 4 0 0 ... 27 24 25]\n [ 1 0 0 ... 26 23...,Training,1,data/Training/Neutral/Neutral-1.jpg,sienna


In [12]:
# Select training data
print(fer2013.shape)
train_df = fer2013[fer2013['usage']=='Training']
print(train_df.shape)

(35887, 9)
(28709, 9)


In [13]:
train_df['emotion'].unique()

array(['Angry', 'Fear', 'Sad', 'Neutral', 'Happy', 'Surprise', 'Disgust'],
      dtype=object)

In [14]:
test_dict = load_json('configs/unsupervised_models_test.json')

In [15]:
print_json(test_dict)

{
    "FastICA": {
        "module": "sklearn.decomposition",
        "class": "FastICA",
        "normalization": "standard",
        "total_components": 100,
        "components_for_reconstruction": [
            1,
            10,
            30,
            50,
            100
        ],
        "params": {
            "algorithm": "parallel",
            "fun": "logcosh",
            "max_iter": 500,
            "tol": 0.0001,
            "random_state": 42
        }
    },
    "NMF": {
        "module": "sklearn.decomposition",
        "class": "NMF",
        "normalization": "none",
        "total_components": 100,
        "components_for_reconstruction": [
            1,
            10,
            30,
            50,
            100
        ],
        "params": {
            "init": "random",
            "solver": "mu",
            "beta_loss": "frobenius",
            "max_iter": 1000,
            "tol": 1e-05,
            "random_state": 42
        }
    },
    "PCA": {
    

In [16]:
print_json(test_dict['PCA'])

{
    "module": "sklearn.decomposition",
    "class": "PCA",
    "normalization": "standard",
    "total_components": 100,
    "components_for_reconstruction": [
        1,
        10,
        30,
        50,
        100
    ],
    "params": {
        "svd_solver": "randomized",
        "whiten": true,
        "random_state": 42
    }
}


In [17]:
X, y = create_X_y(train_df, 'img_path', 'emotion')

In [18]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [19]:
from utils.decomposition_feature_extract import run_single_analysis, generate_analysis_paths

In [20]:
def display_image(image, title=None):
    """
    Display a single 48x48 grayscale image in a Jupyter notebook.
    
    :param image: 1D array (length 2304) or 2D array (48x48)
    :param title: Title for the image (optional)
    """
    # Reshape if necessary
    if image.shape[0] == 2304:
        image = image.reshape(48, 48)
    
    plt.figure(figsize=(5, 5))
    plt.imshow(image, cmap='gray')
    plt.axis('off')
    if title:
        plt.title(title)
    plt.show()

In [21]:
analysis_keys = ['FastICA', 'NMF', 'PCA']

In [22]:
for ak in analysis_keys:
    analysis_config = test_dict[ak]
    analysis_config = generate_analysis_paths(analysis_config)
    analysis_results = run_single_analysis(X, y, analysis_config)

    l_results = analysis_results.tolist()
    for img in l_results[0]['avg_recon_images']:
        display_image(img)
    for img in l_results[5]['avg_recon_images']:
        display_image(img)

Analysis settings:
{
    "module": "sklearn.decomposition",
    "class": "FastICA",
    "normalization": "standard",
    "total_components": 100,
    "components_for_reconstruction": [
        1,
        10,
        30,
        50,
        100
    ],
    "params": {
        "algorithm": "parallel",
        "fun": "logcosh",
        "max_iter": 500,
        "tol": 0.0001,
        "random_state": 42
    },
    "paths": {
        "base_dir": "models/unsupervised",
        "result_dir": "models/unsupervised/fastica_standard_100",
        "log_dir": "models/unsupervised/fastica_standard_100/logs",
        "log_path": "models/unsupervised/fastica_standard_100/logs/log_fastica_standard_100.log",
        "analysis_json": "models/unsupervised/fastica_standard_100/fastica_standard_100_info.json",
        "metrics_file": "models/unsupervised/fastica_standard_100/fastica_standard_100_metrics.csv",
        "avg_reconstructions_file": "models/unsupervised/fastica_standard_100/fastica_avg_reconstruct

/opt/anaconda3/lib/python3.11/site-packages/sklearn/decomposition/_fastica.py:542: FutureWarning: Starting in v1.3, whiten='unit-variance' will be used by default.
  warnings.warn(


Running category: Overall.
Shape of features is: (28709, 100).


/opt/anaconda3/lib/python3.11/site-packages/sklearn/decomposition/_fastica.py:542: FutureWarning: Starting in v1.3, whiten='unit-variance' will be used by default.
  warnings.warn(


Running category: Angry.
Shape of features is: (3995, 100).


...

/opt/anaconda3/lib/python3.11/site-packages/sklearn/decomposition/_fastica.py:542: FutureWarning: Starting in v1.3, whiten='unit-variance' will be used by default.
  warnings.warn(


Running category: Sad.
Shape of features is: (4830, 100).


/opt/anaconda3/lib/python3.11/site-packages/sklearn/decomposition/_fastica.py:542: FutureWarning: Starting in v1.3, whiten='unit-variance' will be used by default.
  warnings.warn(


Running category: Surprise.
Shape of features is: (3171, 100).
Analysis settings saved to models/unsupervised/fastica_standard_100/fastica_standard_100_info.json
Averaged reconstructions saved to models/unsupervised/fastica_standard_100/fastica_avg_reconstructions.npz
Function run_single_analysis Took 266.0888 seconds


KeyError: 'avg_recon_images'